## 数据准备

In [15]:
library(tidyverse)
library(fastDummies)
all_data = read_csv("/Users/dddd1007/project2git/cognitive_control_model/data/input/all_data_with_RL_BL_estimate_result.csv")
all_data_without_error <- all_data %>%
                            group_by(Subject_num, prop, congruency) %>%
                            filter(Type == "hit") %>%
                            filter(abs(RT - mean(RT)) < (sd(RT) * 3))
sub_num_list <- unique(all_data_without_error$Subject_num)
glimpse(all_data_without_error)
mse <- function(sm) 
    mean(sm$residuals^2)

Rows: 30233 Columns: 58
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (10): stim_color, stim_loc, Type, hand, bias, hand_bias_num, Subject, PC...
dbl (48): Response, RT, Trial, prop, prop_factor, condition_factor, block, t...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 29,159
Columns: 58
Groups: Subject_num, prop, congruency [128]
$ stim_color         <chr> "red", "red", "red", "green", "green", "green", "re…
$ stim_loc           <chr> "left", "left", "left", "right", "right", "right", …
$ Type               <chr> "hit", "hit", "hit", "hit", "hit", "hit", "hit", "h…
$ Response           <dbl> 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, …
$ hand               <chr> "left", "left", "left", "right", "right", "right", …
$ bias               <chr> "bias", "bias", "bias", "bias", "bias", "bias", "bi…
$ hand_bias_num      <chr> "0_0", "0_0", "0_0", "1_0", "1_0", "1_0", "0_0", "0…
$ RT                 <dbl> 682.6, 391.8, 404.8, 369.6, 392.5, 325.8, 491.0, 39…
$ Subject            <chr> "sub01_Yangmiao", "sub01_Yangmiao", "sub01_Yangmiao…
$ Trial              <dbl> 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,…
$ prop               <dbl> 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,…
$ prop_factor        <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 

In [11]:
result_list <- list()
sub_count <- 1
for (i in sub_num_list) {
    tmp_data <- filter(all_data_without_error, Subject_num == i)
    mini_num <- 1
    mini_block <- vector(mode = "numeric", length = nrow(tmp_data))
    mini_block[1] <- 1
    for (line_num in 2:nrow(tmp_data)) {
        if (tmp_data$prop[line_num] != tmp_data$prop[line_num - 1]) {
            mini_num <- mini_num + 1
        }
        mini_block[line_num] <- mini_num
    }
    tmp_data$mini_block <- mini_block
    result_list[[sub_count]] <- tmp_data
    sub_count <- sub_count + 1
}
all_data_miniblock <- bind_rows(result_list, .id = "column_label")

## 估计概率与刺激特征之间的关系

In [36]:
# 和刺激特征的比较
aic_bl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab_v <- vector(mode = "numeric", length(sub_num_list))
aic_rl_sr   <- vector(mode = "numeric", length(sub_num_list))

for (i in seq_len(length(sub_num_list))) {
    single_sub_data <- all_data_without_error %>%
                        filter(Subject_num == sub_num_list[i])
    aic_bl_ab[i]   <- AIC(glm(congruency_num ~ bl_ab_r_con, data = single_sub_data, family = binomial))
    # aic_bl_sr[i]   <- AIC(glm(correct_action ~ bl_sr_r_left_hand, data = single_sub_data, family = binomial))
    aic_rl_ab[i]   <- AIC(glm(congruency_num ~ rl_ab_p, data = single_sub_data, family = binomial))
    aic_rl_ab_v[i] <- AIC(glm(congruency_num ~ rl_ab_v_p, data = single_sub_data, family = binomial))
    aic_rl_sr[i]   <- AIC(glm(correct_action ~ rl_sr_p, data = single_sub_data, family = binomial))
}

stim_aic <- data.frame(Subject_num = sub_num_list,
                     bl_ab = aic_bl_ab,
                     rl_ab = aic_rl_ab,
                     rl_ab_v = aic_rl_ab_v,
                     rl_sr = aic_rl_sr)

## 与反应时之间的变量关系

### 添加 Run 作为 截距

In [21]:
# 根据反应时比较 AIC, 此时应当用当前s-r对应的p来拟合
aic_bl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_bl_sr   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_sr   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_sr_v <- vector(mode = "numeric", length(sub_num_list))

for (i in seq_len(length(sub_num_list))) {
    single_sub_data <- all_data_without_error %>%
                        filter(Subject_num == sub_num_list[i])
    single_sub_data$rl_ab_v_pe <- 1 - single_sub_data$rl_ab_p_selected
    single_sub_data$rl_sr_v_pe <- 1 - single_sub_data$rl_sr_v_p_selected

    tmp_data <- dummy_columns(
        filter(
            single_sub_data,
            Subject_num == sub_num_list[i]
        ),
        "run"
    ) %>%
        select(-run)
    bl_ab_data <- select(tmp_data, RT, bl_ab_PE, congruency_num, hand, starts_with("run"))
    model_bl_ab <- lm(RT ~ . -1, data = bl_ab_data)

    bl_sr_data <- select(tmp_data, RT, bl_sr_PE, congruency_num, hand, starts_with("run"))
    model_bl_sr <- lm(RT ~ . -1, data = bl_sr_data)

    rl_ab_data <- select(tmp_data, RT, rl_ab_v_pe, congruency_num, hand, starts_with("run"))
    model_rl_ab <- lm(RT ~ . -1, data = rl_ab_data)

    rl_sr_data <- select(tmp_data, RT, rl_sr_v_pe, congruency_num, hand, starts_with("run"))
    model_rl_sr <- lm(RT ~ . -1, data = rl_sr_data)

    aic_bl_ab[i] <- mse(model_bl_ab)
    aic_bl_sr[i] <- mse(model_bl_sr)
    aic_rl_ab[i] <- mse(model_rl_ab)
    aic_rl_sr[i] <- mse(model_rl_sr)
    
}

RT_aic <- data.frame(Subject_num = sub_num_list,
                     bl_ab = aic_bl_ab,
                     bl_sr = aic_bl_sr,
                     rl_ab = aic_rl_ab,
                     rl_sr = aic_rl_sr)

RT_aic

Subject_num,bl_ab,bl_sr,rl_ab,rl_sr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5127.077,5184.876,5006.080,5119.739
2,7058.248,7023.873,6842.396,6805.599
4,5065.043,5070.463,5018.348,5040.130
5,21263.719,21286.173,21326.422,21430.748
7,6319.639,6332.678,6249.722,6266.922
9,5210.242,5201.070,5150.214,5104.041
10,14592.708,14579.201,14564.272,14559.574
11,14139.060,14224.038,14102.367,14140.712
12,29537.089,29687.708,29264.133,29696.854


In [22]:
summary(model_bl_sr)


Call:
lm(formula = RT ~ . - 1, data = bl_sr_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-121.211  -39.396   -7.159   33.275  183.653 

Coefficients: (1 not defined because of singularities)
               Estimate Std. Error t value Pr(>|t|)    
bl_sr_PE         88.177     11.227   7.854 1.13e-14 ***
congruency_num   -4.650      1.848  -2.516   0.0121 *  
handleft        348.350      6.444  54.058  < 2e-16 ***
handright       350.273      6.418  54.577  < 2e-16 ***
run_1            27.146      6.316   4.298 1.91e-05 ***
run_2            10.023      6.311   1.588   0.1126    
run_3             5.429      6.341   0.856   0.3921    
run_4            12.768      6.325   2.019   0.0438 *  
run_5            15.039      6.295   2.389   0.0171 *  
run_6                NA         NA      NA       NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 55.22 on 914 degrees of freedom
Multiple R-squared:  0.981,	Adjusted R-squared: 